In [21]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from eval_NN import train_eval_save_NN
import os

In [22]:
# TRAIN_CSV = "data/processed/cleaned_data.csv"
# MODEL_PATH = "models/trained/NN_model.pkl"
BASE_DIR=os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
csv_path = os.path.join(BASE_DIR,"data", "processed", "cleaned_data.csv")
outputPath = os.path.join(BASE_DIR,"models", "trained", "NN_model.pkl")

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [24]:

df = pd.read_csv(csv_path)

In [25]:
features = [
        'store_sales(in millions)', 'store_cost(in millions)', 'total_children',
        'avg_cars_at home(approx)', 'num_children_at_home', 'gross_weight',
        'net_weight', 'store_sqft', 'grocery_sqft', 'units_per_case', 'SRP'
    ]
target = 'cost'

In [26]:
data = df[features].values
data_output = df[target].values

In [27]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

data = scaler_X.fit_transform(data)
data_output = scaler_y.fit_transform(data_output.reshape(-1, 1))

In [28]:
data = torch.tensor(data, dtype=torch.float32).to(device)
data_output = torch.tensor(data_output, dtype=torch.float32).to(device).view(-1, 1)

In [ ]:
results = train_eval_save_NN(
    data=data,
    data_output=data_output,
    scaler_y=scaler_y,
    device=device,
    k_folds=5,
    num_epochs=50,
    batch_size=64,
)

Fold 1/5


d:\Anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Epoch [10/50] - Loss: 0.96978
  Epoch [20/50] - Loss: 0.95627
  Epoch [30/50] - Loss: 0.94689
  Epoch [40/50] - Loss: 0.93662
  Epoch [50/50] - Loss: 0.92720
  Accuracy: 0.00000
  Validation MSE: 894.30194
  R² Score: 0.01020
Fold 2/5


In [ ]:
print("\nCross-Validation Results:")
for res in results:
    print(f"Fold {res['fold']}: MSE = {res['mse']:.5f}, R² = {res['r2']:.5f}")


avg_mse = np.mean([r['mse'] for r in results])
avg_r2 = np.mean([r['r2'] for r in results])
print(f"\nAverage MSE: {avg_mse:.5f}")
print(f"Average R²: {avg_r2:.5f}")